# [Module 4] Prophet 및 DeepAR+ Forecast 생성 (Create a Target Forecast)
- 이전 모듈에서 생성한 Predictor를 가지고 Prophet 및 DeepAR+ Forecast 생성을 합니다. Foreast의 의미는 실질적으로 전체 학습을 다시하고 서비스를 할 수 있는 준비를 하는 과정 입니다. Forecast의 생성 이후에 실제 미래의 예측(Inference)을 일괄적으로 하여 S3에 저장을 합니다.

* **About 50 mins may be elapsed**

In [1]:
import boto3
from time import sleep
import pandas as pd
import json
import time
import pprint
import numpy as np

In [2]:
# Recover variables stored by other notebooks
%store -r

In [3]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')

## Create Prophet and DeepAR+ Forecast

In [4]:
# Prophet
prophet_forecastName = project+'_prophet_algo_forecast' + suffix
prophet_create_forecast_response=forecast.create_forecast(
    ForecastName=prophet_forecastName,
    PredictorArn=prophet_predictorArn)
prophet_forecast_arn = prophet_create_forecast_response['ForecastArn']

In [5]:
forecast.describe_forecast(ForecastArn = prophet_forecast_arn)

{'ForecastArn': 'arn:aws:forecast:ap-northeast-2:870180618679:forecast/StoreItemDemand_prophet_algo_forecast88082',
 'ForecastName': 'StoreItemDemand_prophet_algo_forecast88082',
 'ForecastTypes': ['0.1', '0.5', '0.9'],
 'PredictorArn': 'arn:aws:forecast:ap-northeast-2:870180618679:predictor/StoreItemDemand_prophet_algo_188082',
 'DatasetGroupArn': 'arn:aws:forecast:ap-northeast-2:870180618679:dataset-group/StoreItemDemandDSG88082',
 'Status': 'CREATE_PENDING',
 'CreationTime': datetime.datetime(2020, 7, 13, 4, 37, 8, 927000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 7, 13, 4, 37, 8, 927000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'c11fb4d3-9431-457c-8524-8f3fc1a7ff29',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 13 Jul 2020 04:37:10 GMT',
   'x-amzn-requestid': 'c11fb4d3-9431-457c-8524-8f3fc1a7ff29',
   'content-length': '519',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [6]:
# DeepAR+
deeparp_forecastName = project+'_deeparp_algo_forecast' +  suffix
deeparp_create_forecast_response=forecast.create_forecast(
    ForecastName=deeparp_forecastName,
    PredictorArn=deepar_predictorArn)
deeparp_forecast_arn = deeparp_create_forecast_response['ForecastArn']

In [7]:
forecast.describe_forecast(ForecastArn = deeparp_forecast_arn)

{'ForecastArn': 'arn:aws:forecast:ap-northeast-2:870180618679:forecast/StoreItemDemand_deeparp_algo_forecast88082',
 'ForecastName': 'StoreItemDemand_deeparp_algo_forecast88082',
 'ForecastTypes': ['0.1', '0.5', '0.9'],
 'PredictorArn': 'arn:aws:forecast:ap-northeast-2:870180618679:predictor/StoreItemDemand_deeparp_algo_188082',
 'DatasetGroupArn': 'arn:aws:forecast:ap-northeast-2:870180618679:dataset-group/StoreItemDemandDSG88082',
 'Status': 'CREATE_PENDING',
 'CreationTime': datetime.datetime(2020, 7, 13, 4, 37, 11, 422000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 7, 13, 4, 37, 11, 422000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '35dfb01c-4c39-41c4-bb32-812af095dfcf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 13 Jul 2020 04:37:11 GMT',
   'x-amzn-requestid': '35dfb01c-4c39-41c4-bb32-812af095dfcf',
   'content-length': '519',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [9]:
%%time
# Check the Prophet status

while True:
    createProphetStatus = forecast.describe_forecast(ForecastArn= prophet_forecast_arn)['Status']
    createDeeparpStatus = forecast.describe_forecast(ForecastArn= deeparp_forecast_arn)['Status']    
    print("Prophet: ", createProphetStatus)
    print("DeepARP: ", createProphetStatus)     
    if (createProphetStatus != 'ACTIVE' and createProphetStatus != 'CREATE_FAILED') or (createDeeparpStatus != 'ACTIVE' and createDeeparpStatus != 'CREATE_FAILED'):
        sleep(60)    
    else:
        break

Prophet:  ACTIVE
DeepARP:  ACTIVE
CPU times: user 10.5 ms, sys: 1.68 ms, total: 12.2 ms
Wall time: 113 ms


## Upload forecast results to S3
- Forecast를 가지고 일괄적으로 예측 작업을 하여 결과를 S3에 저장 합니다.

In [10]:
prophet_path = "s3://" + bucket_name + "/" + bucket_folder + "/prophet_" + suffix + "/" 
prophet_job_name = "ProphetExport1"+ suffix
create_forecast_export_job_prophet_response = forecast.create_forecast_export_job(
    ForecastExportJobName = prophet_job_name,
    ForecastArn = prophet_forecast_arn,
    Destination={
        "S3Config" : {
            "Path": prophet_path,
            "RoleArn": role_arn
        }
    })


In [11]:
ForecastProphetExportJobArn = create_forecast_export_job_prophet_response["ForecastExportJobArn"]
forecast.describe_forecast_export_job(ForecastExportJobArn = ForecastProphetExportJobArn)

{'ForecastExportJobArn': 'arn:aws:forecast:ap-northeast-2:870180618679:forecast-export-job/StoreItemDemand_prophet_algo_forecast88082/ProphetExport188082',
 'ForecastExportJobName': 'ProphetExport188082',
 'ForecastArn': 'arn:aws:forecast:ap-northeast-2:870180618679:forecast/StoreItemDemand_prophet_algo_forecast88082',
 'Destination': {'S3Config': {'Path': 's3://sagemaker-ap-northeast-2-870180618679/StoreItemDemand/prophet_88082',
   'RoleArn': 'arn:aws:iam::870180618679:role/ForecastRolePOC88082'}},
 'Status': 'CREATE_PENDING',
 'CreationTime': datetime.datetime(2020, 7, 13, 9, 15, 20, 488000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 7, 13, 9, 15, 20, 488000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '0644e7f8-9fe2-42de-819e-fb77b7e01a1b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 13 Jul 2020 09:15:20 GMT',
   'x-amzn-requestid': '0644e7f8-9fe2-42de-819e-fb77b7e01a1b',
   'content-le

In [12]:
deeparp_path = "s3://" + bucket_name + "/" +  bucket_folder +  "/deeparp_"  + suffix + "/" 
deeparp_job_name = "DeepARPExport1" + suffix
create_forecast_export_job_deeparp_response = forecast.create_forecast_export_job(
    ForecastExportJobName =deeparp_job_name,
    ForecastArn = deeparp_forecast_arn,
    Destination={
        "S3Config" : {
            "Path": deeparp_path ,
            "RoleArn": role_arn
        }
    })



In [13]:
ForecastDeeparExportJobArn = create_forecast_export_job_deeparp_response["ForecastExportJobArn"]
forecast.describe_forecast_export_job(ForecastExportJobArn = ForecastDeeparExportJobArn)



{'ForecastExportJobArn': 'arn:aws:forecast:ap-northeast-2:870180618679:forecast-export-job/StoreItemDemand_deeparp_algo_forecast88082/DeepARPExport188082',
 'ForecastExportJobName': 'DeepARPExport188082',
 'ForecastArn': 'arn:aws:forecast:ap-northeast-2:870180618679:forecast/StoreItemDemand_deeparp_algo_forecast88082',
 'Destination': {'S3Config': {'Path': 's3://sagemaker-ap-northeast-2-870180618679/StoreItemDemand/deeparp_88082',
   'RoleArn': 'arn:aws:iam::870180618679:role/ForecastRolePOC88082'}},
 'Status': 'CREATE_PENDING',
 'CreationTime': datetime.datetime(2020, 7, 13, 9, 15, 25, 242000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 7, 13, 9, 15, 25, 242000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '570c3e7f-73ba-4545-a7fb-13b5fc1b1194',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 13 Jul 2020 09:15:25 GMT',
   'x-amzn-requestid': '570c3e7f-73ba-4545-a7fb-13b5fc1b1194',
   'content-le

This exporting process is another one of those items that will take several minutes to complete. Just poll for progress in the console. From the earlier page where you saw the status turn `Active` for a Forecast, click it and you can see the progress of the export.

In [14]:
%%time
# Check the Prophet status

while True:
    createProphetStatus = forecast.describe_forecast_export_job(ForecastExportJobArn= ForecastProphetExportJobArn)['Status']
    createDeeparpStatus = forecast.describe_forecast_export_job(ForecastExportJobArn= ForecastDeeparExportJobArn)['Status']    
    print("Prophet: ", createProphetStatus)
    print("DeepARP: ", createProphetStatus)          
    if (createProphetStatus != 'ACTIVE' and createProphetStatus != 'CREATE_FAILED') or (createDeeparpStatus != 'ACTIVE' and createDeeparpStatus != 'CREATE_FAILED'):
        sleep(60)
        
    else:
        break

Prophet:  CREATE_PENDING
DeepARP:  CREATE_PENDING
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  ACTIVE
DeepARP:  ACTIVE
CPU times: user 57.3 ms, sys: 8.69 ms, total: 66 ms
Wall time: 8min 1s


In [15]:
%store prophet_forecast_arn
%store deeparp_forecast_arn
%store ForecastProphetExportJobArn
%store ForecastDeeparExportJobArn



Stored 'prophet_forecast_arn' (str)
Stored 'deeparp_forecast_arn' (str)
Stored 'ForecastProphetExportJobArn' (str)
Stored 'ForecastDeeparExportJobArn' (str)
